In [10]:
from dotenv import load_dotenv
import os
load_dotenv('../.env_api')
from rich.pretty import pprint
from operator import add
from litellm import completion

def ppprint(obj):
    pprint(obj, indent_guides=False)

os.environ["HTTP_PROXY"] = "http://localhost:9090"
os.environ["HTTPS_PROXY"] = "http://localhost:9090"
os.environ["REQUESTS_CA_BUNDLE"] = "/Users/tomek/Library/Application Support/com.proxyman.NSProxy-setapp/app-data/proxyman-ca.pem"
os.environ["SSL_CERT_FILE"] = "/Users/tomek/Library/Application Support/com.proxyman.NSProxy-setapp/app-data/proxyman-ca.pem"
from langsmith import Client
import os

from typing import List, Annotated, Any, Dict, Optional
from pydantic import BaseModel, Field

import instructor
from openai import OpenAI

import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))


from api.rag.utils.utils import lc_messages_to_regular_messages
from api.rag.utils.utils import prompt_template_config
from api.core.config import config

### Coordinator agent

In [11]:
class MCPToolCall(BaseModel):
    name: str
    arguments: dict
    server: str

class ToolCall(BaseModel):
    name: str
    arguments: dict

class RAGUsedContext(BaseModel):
    id: str
    description: str
    
class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str

class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""

    coordinator_iteration: int = Field(default=0)
    product_qa_iteration: int = Field(default=0)
    shopping_cart_iteration: int = Field(default=0)

    coordinator_final_answer: bool = Field(default=False)
    product_qa_final_answer: bool = Field(default=False)
    shopping_cart_final_answer: bool = Field(default=False)

    product_qa_available_tools: List[Dict[str, Any]] = []
    shopping_cart_available_tools: List[Dict[str, Any]] = []

    tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    mcp_tool_calls: Optional[List[MCPToolCall]] = Field(default_factory=list)
    retrieved_context: List[RAGUsedContext] = Field(default_factory=list)
    
    user_id: str = ""
    cart_id: str = ""

    next_agent: str = ""
    plan: list[Delegation] = Field(default_factory=list)

    trace_id: str = ""

In [12]:
def coordinator_agent_node(state, models = ['gpt-4.1', 'groq/llama-3.3-70b-versatile']) -> dict:
    template = prompt_template_config('../' + config.PROMPT_TEMPLATE_PATH, 'coordinator_agent')
    
    prompt = template.render()

    messages = state.messages

    conversation = []

    for msg in messages:
        conversation.append(lc_messages_to_regular_messages(msg))

    client = instructor.from_litellm(completion)

    for model in models:
        try:
            response, raw_response = client.chat.completions.create_with_completion(
                model=model,
                response_model=CoordinatorAgentResponse,
                messages=[{"role": "system", "content": prompt}, *conversation],
                temperature=0,
            )
            break
        except Exception as e:
            print(f"Error with model {model}: {e}")

    return {
        # "messages": ai_message,
        "next_agent": response.next_agent,
        "plan": response.plan,
        "coordinator_final_answer": response.final_answer,
        "coordinator_iteration": state.coordinator_iteration + 1,
        "answer": response.answer,
        "trace_id": ''
    }

In [13]:
initial_state = State(messages=[    
    {"role": "user", "content": "What is the weather in Tokyo?"}
])

#answer = coordinator_agent_node(initial_state, models=['groq/llama-3.3-70b-versatile'])
answer = coordinator_agent_node(initial_state)

In [14]:
pprint(answer)

{
│   'next_agent': '',
│   'plan': [],
│   'coordinator_final_answer': True,
│   'coordinator_iteration': 1,
│   'answer': "I'm here to assist you with shopping-related questions, such as finding products, checking specifications, or managing your shopping cart. I don't have access to real-time weather information. If you have any questions about products or shopping, feel free to ask!",
│   'trace_id': ''
}